In [5]:
import pandas as pd
import numpy as np
import seaborn as sns
import datetime
import math
import quandl
import collections, functools, operator
import matplotlib.pyplot as plt  
import seaborn as seabornInstance
import statsmodels.api as sm
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from operator import itemgetter
pd.set_option('display.max_columns', 500)
pd.set_option('mode.chained_assignment', None)
%matplotlib inline

In [2]:
test = pd.read_csv('test_output.csv')
train = pd.read_csv('train_output.csv')
train = train.drop(columns = ['Unnamed: 0'])    
test = test.drop(columns = ['Unnamed: 0'])    

In [16]:
data_train = train.copy()

# conventional weight
data_train['conv_weight_temp'] = data_train.groupby(['YearMonth'])['ticker_ret'].apply(lambda x: (x-x.mean()))
data_train['conv_weight_temp1'] = data_train['conv_weight_temp'].apply(lambda x: 0 if x<0 else x)
data_train['conv_weight_temp1'] = data_train['conv_weight_temp1'].groupby(data_train['YearMonth']).transform('sum')
data_train['conv_weight'] = data_train['conv_weight_temp']/data_train['conv_weight_temp1']

#residual weight
data_train['residual_weight_temp'] = data_train.groupby(['YearMonth'])['residual'].apply(lambda x: (x-x.mean()))
data_train['residual_weight_temp1'] = data_train['residual_weight_temp'].apply(lambda x: 0 if x<0 else x)
data_train['residual_weight_temp1'] = data_train['residual_weight_temp1'].groupby(data_train['YearMonth']).transform('sum')
data_train['residual_weight'] = data_train['residual_weight_temp']/data_train['residual_weight_temp1']



data_train = data_train[['YearMonth','PERMNO','conv_weight','residual_weight']]

In [24]:
def changemonth(yearmonth):
    year = str(yearmonth)[:4]
    month = str(yearmonth)[-2:]
    if int(month)+1 < 10:
        month = '0'+str(int(month)+1)
    elif int(month)+1 > 12:
        month = '01'
        year = str(int(year)+1)
    else:
        month = str(int(month)+1)
    return year+month

In [27]:
data_train['YearMonth'] = data_train['YearMonth'].apply(lambda x: changemonth(x))

In [42]:
test['YearMonth'] = test['YearMonth'].astype(int)

In [43]:
test_output = pd.merge(data_train, test, on = ['PERMNO','YearMonth'], how = 'right')

(536300, 15)

In [47]:
data_train.shape

(536300, 4)

In [48]:
test.shape

(536300, 13)

In [50]:
test_output

,YearMonth,PERMNO,conv_weight,residual_weight,CUSIP,EXCHCD,PRC,RET,RETX,SHRCD,SHROUT,TICKER,date,market cap,mkt_cap
0,192908,12386,-0.025879,-0.014615,44731210,1.0,40.37500,-0.024169,-0.024169,10.0,1165.0,NaN,1929-08-30,26430000,4.703688e+04
1,192908,10030,0.001763,0.001141,00338690,1.0,59.00000,0.030568,0.030568,10.0,649.0,NaN,1929-08-30,26430000,3.829100e+04
2,192908,13864,0.014535,0.009581,71363590,1.0,243.00000,0.221106,0.221106,10.0,450.0,NaN,1929-08-30,26430000,1.093500e+05
3,192908,11068,0.023972,0.018679,14742910,1.0,345.00000,-0.057377,-0.057377,10.0,130.0,NaN,1929-08-30,26430000,4.485000e+04
4,192908,10890,-0.006811,-0.010559,90921430,1.0,70.50000,0.111635,0.108491,11.0,5000.0,NaN,1929-08-30,26430000,3.525000e+05
5,192908,15595,0.015746,0.013643,92599090,1.0,49.00000,-0.033292,-0.033292,10.0,800.0,NaN,1929-08-30,26430000,3.920000e+04
6,192908,10428,0.001443,0.005867,03023190,1.0,200.00000,0.134831,0.123596,11.0,805.0,NaN,1929-08-30,26430000,1.610000e+05
7,192908,13311,0.003201,0.005145,62952710,1.0,213.00000,0.075758,0.075758,10.0,2210.0,NaN,1929-08-30,26430000,4.707300e+05
8,192908,14568,0.019432,0.016595,85399090,1.0,163.50000,0.220149,0.220149,10.0,1561.0,NaN,1929-08-30,26430000,2.552235e+05
9,192908,14066,-0.002783,-0.001902,74579110,1.0,86.00000,-0.007215,-0.007215,10.0,3376.0,NaN,1929-08-30,26430000,2.903360e+05
